* Generate weight matrix + cov_a & send to Bruno as a mat file

* Plot eigenvectors of cov_a in addition to pooling weights

* Why are there no negative eigenvectors?

* Create summary plot for cov_a

* rerun everything with ICA vs LCA

* Aapo does not subsample
  * If you reduce to K dimensions, then you technically should only need K pooling filters
  * How to space them evenly?

* Rewrite pooling as a 3-term energy function, write LCA to get feedback from pooling layer
  * could also get feedback from eigenvectors
 
* Once you have a 2-layer network, you should be able to compress or denoise an image

* Write up that discussess all pooling methods in one framework? I've started this...

* For get_dictionary_stats: Should use center of mass of envelope, instead of max pixel value.
  * Upsample fft / ifft for finding a better center
  * This prevents issues with aliasing. You're limited to 16x16px resolution, but the true center of the BF could be between pixels.
  * Convolving with Laplacian will make it slightly more peaky, but will not solve the aliasing problem.
  * Possible methods could be fitting Gaussian, computing convex hull, computing blob boundaries.
  * Add zero padding option to hilbertize
    * Can zero-pad in space domain to get interpolated fft
    * can zero-pad (e.g. 16x16 -> 64x64) in fft domain, then ifft, to get interpolated space image
    * once you get centers, multiply by e.g. 64/16
  * Finding centers
    * center of mass of (upsampled?) thresholded image
    * fit a gaussian
  
* For a given row in covariance matrix, plot that BF, then plot the BF corresponding to the top N indices in the row. These should be visually similar to the row's BF.

## Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import cv2
import os                                                                       
import skimage.draw
import numpy as np                                                              
import tensorflow as tf                                                         
                                                                                
import data.data_picker as dp                                                   
import utils.plot_functions as pf                                               
import utils.image_processing as ip                                             

## Model parameters

In [ ]:
params = {                                                                      
  ## Model params                                                               
  "out_dir": os.path.expanduser("~")+"/Work/Projects/strongPCA/outputs/",       
  "chk_dir": os.path.expanduser("~")+"/Work/Projects/strongPCA/checkpoints/",
  #"data_dir": "/media/tbell/datasets/", #for remote machine
  "data_dir": os.path.expanduser("~")+"/Work/Datasets/", #for local runs                       
  "load_chk": False,                                                             
  "update_interval": 100,                                                       
  "device": "/cpu:0",                                                           
  "learning_rate": 0.12,                                                        
  "num_neurons": 500,                                                           
  "sparse_mult": 0.2,                                                           
  "num_inference_steps": 20,                                                    
  "eta": 0.001/0.03, #dt/tau                                                    
  "eps": 1e-12,                                                                 
  ## Data params                                                                
  "data_type": "vanhateren",                                                    
  "rand_state": np.random.RandomState(12345),                                   
  "num_images": 500,                                                          
  "num_batches": int(1e6),                                                         
  "batch_size": 100,                                                            
  "patch_edge_size": 16,                                                        
  "overlapping_patches": True,                                                  
  "patch_variance_threshold": 1e-6,                                             
  "conv": False,                                                                
  "whiten_images": True}                                                        
                                                                                
## Calculated params                                                            
params["epoch_size"] = params["batch_size"] * params["num_batches"]             
params["num_pixels"] = int(params["patch_edge_size"]**2)                        
params["dataset_shape"] = [int(val)                                             
    for val in [params["epoch_size"], params["num_pixels"]]],                   
params["phi_shape"] = [params["num_pixels"], params["num_neurons"]]             

## Load dataset

In [ ]:
data = dp.get_data(params["data_type"], params)                                 
params["input_shape"] = [                                                       
  data["train"].num_rows*data["train"].num_cols*data["train"].num_channels]     

## Define graph

In [ ]:
graph = tf.Graph()                                                              
with tf.device(params["device"]):                                               
  with graph.as_default():                                                      
    with tf.name_scope("placeholders") as scope:                                
      x = tf.placeholder(tf.float32, shape=[None,               
        params["num_pixels"]], name="input_data")                               
      sparse_mult = tf.placeholder(tf.float32, shape=(), name="sparse_mult")    

    with tf.name_scope("constants") as scope:                                   
      u_zeros = tf.zeros(shape=tf.stack([tf.shape(x)[0],                        
        params["num_neurons"]]), dtype=tf.float32, name="u_zeros")              

    with tf.name_scope("step_counter") as scope:                                
      global_step = tf.Variable(0, trainable=False, name="global_step")         

    with tf.variable_scope("weights") as scope:                                 
      phi_init = tf.truncated_normal(params["phi_shape"], mean=0.0, stddev=0.2, 
        dtype=tf.float32, name="phi_init")                                  
      phi = tf.get_variable(name="phi", dtype=tf.float32,                       
        initializer=phi_init, trainable=True)                                   

    with tf.name_scope("norm_weights") as scope:                                
      norm_phi = phi.assign(tf.nn.l2_normalize(phi,                             
        dim=0, epsilon=params["eps"], name="row_l2_norm"))                      
      norm_weights = tf.group(norm_phi,                                         
        name="l2_normalization")                                                

    with tf.name_scope("inference") as scope:                                   
      u = tf.Variable(u_zeros, trainable=False,                                 
        validate_shape=False, name="u")                                         
      # soft thresholded, rectified                                             
      a = tf.where(tf.greater(u, sparse_mult),                                  
        tf.subtract(u, sparse_mult), u_zeros,                                   
        name="activity")                                                        

    with tf.name_scope("output") as scope:                                      
      with tf.name_scope("image_estimate"):                                     
        x_ = tf.matmul(a, tf.transpose(phi),                                    
          name="reconstruction")                                                

    with tf.name_scope("loss") as scope:                                        
      with tf.name_scope("unsupervised"):                                       
        recon_loss = tf.reduce_mean(0.5 *                                       
          tf.reduce_sum(tf.pow(tf.subtract(x, x_), 2.0),                        
          axis=[1]), name="recon_loss")                                         
        sparse_loss = sparse_mult * tf.reduce_mean(                             
          tf.reduce_sum(tf.abs(a), axis=[1]),                                   
          name="sparse_loss")                                                   
        unsupervised_loss = (recon_loss + sparse_loss)                          
      total_loss = unsupervised_loss                                            

    with tf.name_scope("update_u") as scope:                                    
      lca_b = tf.matmul(x, phi, name="driving_input")                           
      lca_g = (tf.matmul(tf.transpose(phi), phi,                                
        name="gram_matrix") -                                                   
        tf.constant(np.identity(params["phi_shape"][1], dtype=np.float32),      
        name="identity_matrix"))                                                
      lca_explain_away = tf.matmul(a, lca_g,                                    
        name="explaining_away")                                                 
      du = lca_b - lca_explain_away - u                                         
      step_inference = tf.group(u.assign_add(params["eta"] * du),               
        name="step_inference")                                                  
      reset_activity = tf.group(u.assign(u_zeros),                              
        name="reset_activity")                                                  

    with tf.name_scope("performance_metrics") as scope:                         
      with tf.name_scope("reconstruction_quality"):                             
        MSE = tf.reduce_mean(tf.pow(tf.subtract(x, x_), 2.0),                   
          axis=[1, 0], name="mean_squared_error")                               

    with tf.name_scope("optimizers") as scope:                                  
      learning_rates = tf.train.exponential_decay(                              
        learning_rate=params["learning_rate"],                                  
        global_step=global_step,                                                
        decay_steps=int(np.floor(params["num_batches"]*0.8)),                   
        decay_rate=0.5,                                                         
        staircase=True,                                                         
        name="phi_annealing_schedule")                                          
      optimizer = tf.train.GradientDescentOptimizer(learning_rates,             
        name="phi_optimizer")                                                   
      update_weights = optimizer.minimize(total_loss, global_step=global_step, 
        var_list=[phi], name="phi_minimizer")                                   

    full_saver = tf.train.Saver(var_list=[phi], max_to_keep=2)                  

    with tf.name_scope("summaries") as scope:                                   
      #tf.summary.image("input", tf.reshape(x, [params["batch_size"],           
      #  params["patch_edge_size"], params["patch_edge_size"], 1]))             
      #tf.summary.image("weights", tf.reshape(tf.transpose(phi),                
      #  [params["num_neurons"], params["patch_edge_size"],                     
      #  params["patch_edge_size"], 1]))                                        
      tf.summary.histogram("u", u)                                              
      tf.summary.histogram("a", a)                                              
      tf.summary.histogram("phi", phi)                                          

    merged_summaries = tf.summary.merge_all()                                   
    train_writer = tf.summary.FileWriter(params["out_dir"], graph)              

    with tf.name_scope("initialization") as scope:                              
      init_op = tf.group(tf.global_variables_initializer(),                     
        tf.local_variables_initializer())                                       

## Train LCA model (if needed)

In [ ]:
if not params["load_chk"]:
  if not os.path.exists(params["out_dir"]):
    os.makedirs(params["out_dir"])
  if not os.path.exists(params["chk_dir"]):
    os.makedirs(params["chk_dir"])

  with tf.Session(graph=graph) as sess:
    sess.run(init_op,
    feed_dict={x:np.zeros([params["batch_size"]]+params["input_shape"],
      dtype=np.float32)})
    
    batch_steps = []
    losses = []
    sparsities = []
    recon_errors = []
    for b_step in range(params["num_batches"]):
      data_batch = data["train"].next_batch(params["batch_size"])
      input_data = data_batch[0]
    
      feed_dict = {x:input_data, sparse_mult:params["sparse_mult"]}
    
      sess.run(norm_weights, feed_dict)
    
      for inference_step in range(params["num_inference_steps"]):               
        sess.run(step_inference, feed_dict)
    
      sess.run(update_weights, feed_dict)
    
      current_step = sess.run(global_step)
      if (current_step % params["update_interval"] == 0):
        summary = sess.run(merged_summaries, feed_dict)
        train_writer.add_summary(summary, current_step)
        full_saver.save(sess, save_path=params["chk_dir"]+"lca_chk",
          global_step=global_step)
    
        [current_loss, a_vals, recons, recon_err, weights] = sess.run(
          [total_loss, a, x_, MSE, phi], feed_dict)
        a_vals_max = np.array(a_vals.max()).tolist()
        a_frac_act = np.array(np.count_nonzero(a_vals)
          / float(params["batch_size"]*params["num_neurons"])).tolist()
        batch_steps.append(current_step)
        losses.append(current_loss)
        sparsities.append(a_frac_act)
        recon_errors.append(recon_err)
    
        print_dict = {"current_step":str(current_step).zfill(5),
          "loss":str(current_loss),
          "a_max":str(a_vals_max),
          "a_frac_act":str(a_frac_act)}
        print(print_dict)
        pf.save_data_tiled(weights.T.reshape((params["num_neurons"],
          params["patch_edge_size"], params["patch_edge_size"])),
          normalize=False, title="Dictionary at step "+str(current_step),
          save_filename=(params["out_dir"]+"phi_"+str(current_step).zfill(5)
          +".png"))
        pf.save_data_tiled(recons.reshape((params["batch_size"],
          params["patch_edge_size"], params["patch_edge_size"])),
          normalize=False, title="Recons at step "+str(current_step),
          save_filename=(params["out_dir"]+"recons_"
          +str(current_step).zfill(5)+".png"))
      output_data = {"batch_step":batch_steps, "total_loss":losses,
        "frac_active":sparsities, "recon_MSE":recon_errors}
      pf.save_stats(output_data, save_filename=(params["out_dir"]+"loss.png"))

## Load large image dataset for PCA analysis

### Incremental covariance matrix function

In [ ]:
def compute_incremental_cov(values, prev_cov=None):
  if prev_cov is None:
    return np.cov(values)
  else:
    return prev_cov + np.cov(values)

### Pooling filters function

In [ ]:
def compute_pooling_filters(a_cov, num_pooling_dims):
  evals, evecs = np.linalg.eig(a_cov)
  sort_indices = np.argsort(evals)[::-1]
  top_vecs = evecs[sort_indices[:num_pooling_dims]]
  pooling_filters = np.dot(top_vecs.T, top_vecs)
  return pooling_filters

### Compute activity covariance matrix

In [ ]:
num_images = 1e6
batch_size = int(1e2)
a_cov = None
with tf.Session(graph=graph) as sess:
  sess.run(init_op,
    feed_dict={x:np.zeros([batch_size]+params["input_shape"],
    dtype=np.float32)})
  full_saver.restore(sess, tf.train.latest_checkpoint(params["chk_dir"]))
  tot_images = 0
  while tot_images < num_images: 
    input_data = data["train"].next_batch(batch_size)[0]
    feed_dict = {x:input_data, sparse_mult:params["sparse_mult"]}
    for inference_step in range(params["num_inference_steps"]):               
      sess.run(step_inference, feed_dict)
    [a_vals, weights] = sess.run([a, phi], feed_dict)
    a_cov = compute_incremental_cov(a_vals.T, a_cov) 
    tot_images += batch_size

### Compute pooling filters

In [ ]:
num_pooling_dims = 50 #K
pooling_filters = compute_pooling_filters(a_cov, num_pooling_dims)

## Functions for analysis

### Hilbertize

In [ ]:
def hilbertize(dictionary, params):
  cart2pol = lambda x,y: (np.arctan2(y,x), np.hypot(x, y))
  
  # Amount of zero padding for fft2 (closest power of 2)                          
  N = np.int(2**(np.ceil(np.log2(params["patch_edge_size"]))))                                  
                                                                                  
  # Analytic signal envelope for dictionary                                                
  # (Hilbet transform of each basis function)                                     
  env = np.zeros((params["num_neurons"], params["num_pixels"]), dtype=complex)                        
                                                                                  
  # Fourier transform of dictionary                                               
  bffs = np.zeros((params["num_neurons"], N, N), dtype=complex)                               
  # Filtered Fourier transform of dictionary                                               
  bff_filt = np.zeros((params["num_neurons"], N**2), dtype=complex)                               

  # Hilbert filters
  hil_filt = np.zeros((params["num_neurons"], params["patch_edge_size"],
    params["patch_edge_size"]))

  # Grid for creating filter
  f = (2/N) * np.pi * np.arange(-N/2.0, N/2.0)                                    
  (fx, fy) = np.meshgrid(f, f)                                                    
  (theta, r) = cart2pol(fx, fy)                                                    
                                                                                  
  for neuron_idx in range(params["num_neurons"]):                                           
    # Grab single basis function, reshape to a square image                       
    bf = dictionary[:, neuron_idx].reshape(params["patch_edge_size"],
      params["patch_edge_size"])               
                                                                                  
    # Convert basis function into DC-centered Fourier domain                      
    bff = np.fft.fftshift(np.fft.fft2(bf, [N, N]))
    bffs[neuron_idx, ...] = bff
                                                                                  
    # Find indices of the peak amplitude                                          
    max_ys = np.abs(bff).argmax(axis=0) # Returns row index for each col          
    max_x = np.argmax(np.abs(bff).max(axis=0))                                    
                                                                                  
    # Convert peak amplitude location into angle in freq domain                   
    fx_ang = f[max_x]                                                             
    fy_ang = f[max_ys[max_x]]                                                     
    theta_max = np.arctan2(fy_ang, fx_ang)                                        
                                                                                  
    # Define the half-plane with respect to the maximum                           
    ang_diff = np.abs(theta-theta_max)                                            
    idx = (ang_diff>np.pi).nonzero()                                              
    ang_diff[idx] = 2.0 * np.pi - ang_diff[idx]                                   
    hil_filt[neuron_idx, ...] = (ang_diff < np.pi/2.0).astype(int)                                        
                                                                                  
    # Create analytic signal from the inverse FT of the half-plane filtered bf
    abf = np.fft.ifft2(np.fft.fftshift(hil_filt[neuron_idx, ...]*bff))                                  
    env[neuron_idx, ...] = abf[0:params["patch_edge_size"],
      0:params["patch_edge_size"]].reshape(params["num_pixels"])    
    bff_filt[neuron_idx, ...] = (hil_filt[neuron_idx, ...]*bff).reshape(N**2)                                     
  return (env, bff_filt, hil_filt, bffs)

### Get dictionary statistics

In [ ]:
def get_dictionary_stats(dictionary, params):
  envelope, bff_filt, hil_filter, bffs = hilbertize(dictionary, params)
  
  basis_funcs = []
  envelopes = []
  filters = []
  envelope_centers = []
  fourier_centers = []
  fourier_maps = []
  orientations = []
  lengths = []
  line_images = []
  blob_images = []
  for bf_idx in range(params["num_neurons"]):
    # Reformatted individual basis function
    basis_funcs.append(np.squeeze(ip.reshape_data(dictionary.T[bf_idx,...],
      flatten=False)[0]))
  
    # Reformatted individual envelope filter
    envelopes.append(np.squeeze(ip.reshape_data(np.abs(envelope[bf_idx,...]),
      flatten=False)[0]))

    # Basis function center
    max_ys = envelopes[bf_idx].argmax(axis=0) # Returns row index for each col          
    max_x = np.argmax(envelopes[bf_idx].max(axis=0))
    y_cen = max_ys[max_x]
    x_cen = max_x
    envelope_centers.append((y_cen, x_cen)) 

    # Basis function orientation
    filt = hil_filter[bf_idx, ...]
    filters.append(filt)
    y, x = np.nonzero(filt)
    x = x - np.mean(x)
    y = y - np.mean(y)
    coords = np.vstack([y, x])
    evals, evecs = np.linalg.eigh(np.cov(coords))
    sort_indices = np.argsort(evals)[::-1]
    filt_evec = evecs[:, sort_indices[0]]
    orientations.append(filt_evec)

    # Basis function length
    env_evals, env_evecs = np.linalg.eigh(np.cov(envelopes[bf_idx])) 
    sorted_indices = np.argsort(env_evals)[::-1]
    env_major_length = env_evals[sort_indices[0]]
    env_minor_length = env_evals[sort_indices[1]]
    lengths.append((env_major_length, env_minor_length))

    # Rastered basis function line representation
    out_image = np.zeros_like(basis_funcs[bf_idx]) # row (y), col (x)
    y_start = np.int32(np.max([0, np.min([np.ceil(y_cen-filt_evec[0]*lengths[bf_idx][0]), 15])]))
    x_start = np.int32(np.max([0, np.min([np.ceil(x_cen-filt_evec[1]*lengths[bf_idx][0]), 15])]))
    y_end = np.int32(np.max([0, np.min([np.ceil(y_cen+filt_evec[0]*lengths[bf_idx][0]), 15])]))
    x_end = np.int32(np.max([0, np.min([np.ceil(x_cen+filt_evec[1]*lengths[bf_idx][0]), 15])]))
    y_lin, x_lin = skimage.draw.line(y_start, x_start, y_end, x_end)
    out_image[y_lin, x_lin] += 1
    line_images.append(out_image)

    # Rastered basis function blob representations
    #out_image = np.zeros_like(basis_funcs[bf_idx]) # row (y), col (x)
    #rot = np.arctan2(orientations[bf_idx][0], orientations[bf_idx][1])
    #y_elip, x_elip = skimage.draw.ellipse(y_cen, x_cen, env_major_length,
    #  env_minor_length, rotation=rot)
    #out_image[y_elip, x_elip] += 1
    thr_env = envelopes[bf_idx].copy()
    thr_env[np.where(thr_env<np.mean(thr_env)+2*np.std(thr_env))] = 0
    #out_image = np.zeros_like(basis_funcs[bf_idx]) # row (y), col (x)
    #env_y, env_x = np.nonzero(thr_env)
    #points = np.hstack([env_y, env_x])
    #import scipy.spatial
    #hull = scipy.spatial.ConvexHull(points)
    #out_image[points[hull.vertices,0], points[hull.vertices,1]] += 1
    blob_images.append(thr_env)
#
    # Fourier function center
    fourier_map = np.sqrt(np.real(bffs[bf_idx, ...])**2+np.imag(bffs[bf_idx, ...])**2)
    fourier_maps.append(fourier_map)
    max_fys = fourier_map.argmax(axis=0)
    max_fx = np.argmax(fourier_map.max(axis=0))
    fy_cen = max_ys[max_x]
    fx_cen = max_x
    fourier_centers.append((fy_cen, fx_cen))
    
  output = {"basis_functions":basis_funcs, "envelopes":envelopes,
    "filters":filters, "envelope_centers":envelope_centers, "lengths":lengths,
    "fourier_centers":fourier_centers, "fourier_maps":fourier_maps,
    "orientations":orientations, "line_images":line_images,
    "blob_images":blob_images}
  return output

## Functions for plotting

### Plot hilbert analysis

In [ ]:
def plot_hilbert_analysis(weights, params):
  Envelope, bff_filt, Hil_filter, bff = hilbertize(weights, params)
  N = np.int32(np.sqrt(bff_filt.shape[1]))
                                                                                  
  # Plot BFs, Hilbert filtered fourier amplitudes, raw fourier amplitudes
  fig, sub_axes = plt.subplots(3, 1, figsize=(64,64))
  
  plot_data = pf.pad_data(weights.T.reshape((params["num_neurons"],
    params["patch_edge_size"], params["patch_edge_size"])))
  bf_axis_image = sub_axes[0].imshow(plot_data, cmap="Greys_r",
    interpolation="nearest")
  sub_axes[0].tick_params(axis="both", bottom="off", top="off", left="off",
    right="off")
  sub_axes[0].get_xaxis().set_visible(False)
  sub_axes[0].get_yaxis().set_visible(False)
  sub_axes[0].set_title("Basis Functions", fontsize=32)
  
  plot_data = pf.pad_data(np.abs(Envelope).reshape((params["num_neurons"],
    params["patch_edge_size"], params["patch_edge_size"])))
  hil_axis_image = sub_axes[1].imshow(plot_data, cmap="Greys_r",
    interpolation="nearest")
  sub_axes[1].tick_params(axis="both", bottom="off", top="off", left="off",
    right="off")
  sub_axes[1].get_xaxis().set_visible(False)
  sub_axes[1].get_yaxis().set_visible(False)
  sub_axes[1].set_title("Analytic Signal Amplitude Envelope", fontsize=32)
  
  resh_Zf = np.abs(bff_filt).reshape((params["num_neurons"], N, N))                             
  output_z = np.zeros(resh_Zf.shape)                                              
  for i in range(params["num_neurons"]):                                                    
    output_z[i,...] = resh_Zf[i,...] / np.max(resh_Zf[i,...])                     
  plot_data = pf.pad_data(output_z)
  hil_axis_image = sub_axes[2].imshow(plot_data, cmap="Greys_r",
    interpolation="nearest")
  sub_axes[2].tick_params(axis="both", bottom="off", top="off", left="off",
    right="off")
  sub_axes[2].get_xaxis().set_visible(False)
  sub_axes[2].get_yaxis().set_visible(False)
  sub_axes[2].set_title("Fourier Amplitude Spectrum", fontsize=32)
  plt.show()

### Plot basis function statistics

In [ ]:
def plot_bf_stats(weights, params):
    bf_stats = get_dictionary_stats(weights, params)

    num_bf = 10
    fig, sub_axes = plt.subplots(num_bf, 6, figsize=(36,36))
    for bf_idx in range(num_bf):
      bf = bf_stats["basis_functions"][bf_idx]
      env = bf_stats["envelopes"][bf_idx]
      filt = bf_stats["filters"][bf_idx]
      fourier = bf_stats["fourier_maps"][bf_idx]
      y, x = bf_stats["orientations"][bf_idx]
      y_cen, x_cen = bf_stats["envelope_centers"][bf_idx]
      length = bf_stats["lengths"][bf_idx] 
      line_img = bf_stats["line_images"][bf_idx]
      blob_img = bf_stats["blob_images"][bf_idx]
      fy_cen, fx_cen = bf_stats["fourier_centers"][bf_idx]

      sub_axes[bf_idx, 0].imshow(bf, cmap="Greys_r", interpolation="Nearest")
      sub_axes[bf_idx, 0].tick_params(axis="both", bottom="off", top="off",
        left="off", right="off")
      sub_axes[bf_idx, 0].get_xaxis().set_visible(False)
      sub_axes[bf_idx, 0].get_yaxis().set_visible(False)

      sub_axes[bf_idx, 1].imshow(env, cmap="Greys_r", interpolation="Nearest")
      sub_axes[bf_idx, 1].tick_params(axis="both", bottom="off", top="off",
        left="off", right="off")
      sub_axes[bf_idx, 1].get_xaxis().set_visible(False)
      sub_axes[bf_idx, 1].get_yaxis().set_visible(False)

      sub_axes[bf_idx, 2].imshow(filt, cmap="Greys_r", interpolation="Nearest")
      sub_axes[bf_idx, 2].tick_params(axis="both", bottom="off", top="off",
        left="off", right="off")
      sub_axes[bf_idx, 2].get_xaxis().set_visible(False)
      sub_axes[bf_idx, 2].get_yaxis().set_visible(False)

      sub_axes[bf_idx, 3].imshow(fourier, cmap="Greys_r", interpolation="Nearest")
      sub_axes[bf_idx, 3].tick_params(axis="both", top="off", right="off",
        bottom="off", left="off")
      sub_axes[bf_idx, 3].spines["left"].set_position("center")
      sub_axes[bf_idx, 3].spines["left"].set_color("black")
      sub_axes[bf_idx, 3].spines["left"].set_linewidth(2.5)
      sub_axes[bf_idx, 3].spines["bottom"].set_position("center")
      sub_axes[bf_idx, 3].spines["bottom"].set_color("black")
      sub_axes[bf_idx, 3].spines["bottom"].set_linewidth(2.5)
      sub_axes[bf_idx, 3].spines["top"].set_color("none")
      sub_axes[bf_idx, 3].spines["right"].set_color("none")
      sub_axes[bf_idx, 3].set_yticklabels([])
      sub_axes[bf_idx, 3].set_xticklabels([])
      sub_axes[bf_idx, 3].set_ylim([0, fourier.shape[0]-1])
      sub_axes[bf_idx, 3].set_xlim([0, fourier.shape[1]-1])

      line_img[y_cen, x_cen] += 1
      sub_axes[bf_idx, 4].imshow(line_img, interpolation="Nearest")
      sub_axes[bf_idx, 4].tick_params(axis="both", bottom="off", top="off",
        left="off", right="off")
      sub_axes[bf_idx, 4].get_xaxis().set_visible(False)
      sub_axes[bf_idx, 4].get_yaxis().set_visible(False)
        
      blob_img[np.where(blob_img>0)] = 1
      blob_img[y_cen, x_cen] += 1
      sub_axes[bf_idx, 5].imshow(blob_img, interpolation="Nearest")
      sub_axes[bf_idx, 5].tick_params(axis="both", bottom="off", top="off",
        left="off", right="off")
      sub_axes[bf_idx, 5].get_xaxis().set_visible(False)
      sub_axes[bf_idx, 5].get_yaxis().set_visible(False)

    sub_axes[0,0].set_title("bf", fontsize=32)
    sub_axes[0,1].set_title("envelope", fontsize=32)
    sub_axes[0,2].set_title("filter", fontsize=32)
    sub_axes[0,3].set_title("fourier map", fontsize=32)
    sub_axes[0,4].set_title("envelope summary line", fontsize=22)
    sub_axes[0,5].set_title("envelope summary blob", fontsize=22)

    plt.show()

### Plot pooling filter images

In [ ]:
def plot_pooling_filter_images(params, weights, pooling_filters,
  num_connected_weights, num_pooling_filters):
    
  bf_stats = get_dictionary_stats(weights, params)
  
  out_images = np.zeros((num_pooling_filters, params["patch_edge_size"],
    params["patch_edge_size"]))
  for filter_idx in range(num_pooling_filters):
    example_filter = pooling_filters[filter_idx,:]
    top_indices = np.argsort(example_filter)[::-1]
  
    out_image = np.zeros((params["patch_edge_size"], params["patch_edge_size"]))
    for bf_idx in range(num_connected_weights):
      top_idx = top_indices[bf_idx]
      bot_idx = top_indices[::-1][bf_idx]
      top_line_indices = np.nonzero(bf_stats["images"][top_idx])
      bottom_line_indices = np.nonzero(bf_stats["images"][bot_idx])
      out_image[top_line_indices] = np.real(evals[top_idx])
      out_image[bottom_line_indices] = np.real(evals[bot_idx]) 
    out_images[filter_idx, ...] = out_image
  plot_data = pf.pad_data(out_images, pad_values=0)
  
  fig, sub_axes = plt.subplots(1, figsize=(10,10))
  img_ax = sub_axes.imshow(plot_data, cmap="bwr", vmin=-1, vmax=1,
    interpolation="nearest")
  sub_axes.tick_params(axis="both", bottom="off", top="off", left="off",
    right="off")
  sub_axes.get_xaxis().set_visible(False)
  sub_axes.get_yaxis().set_visible(False)
  cbar = fig.colorbar(img_ax, ticks=[-1, 0, 1])
  cbar.ax.tick_params(labelsize=24)
  plt.show()

### Generalized plotting functions

In [ ]:
def plot_pooling_func(plot_func, params, weights, pooling_filters,
  num_connected_weights, num_pooling_filters):

  bf_stats = get_dictionary_stats(weights, params)

  cmap = plt.get_cmap('bwr')
  cNorm = matplotlib.colors.Normalize(vmin=-1, vmax=1)
  scalarMap = matplotlib.cm.ScalarMappable(norm=cNorm, cmap=cmap)

  num_plts_y = np.int32(np.ceil(np.sqrt(num_pooling_filters)))
  num_plts_x = np.int32(np.floor(np.sqrt(num_pooling_filters)))+1 # +cbar row

  print(pooling_filters.shape[0])
  fig, sub_axes = plt.subplots(num_plts_y, num_plts_x, figsize=(24,24))
  filter_idx_list = np.random.choice(np.arange(pooling_filters.shape[0],
    dtype=np.int32), size=num_pooling_filters, replace=False)
  filter_total = 0
  for y_id in range(num_plts_y):
    for x_id in range(num_plts_x):
      if (filter_total < num_pooling_filters and x_id != num_plts_x-1):
        filter_idx = filter_idx_list[filter_total]
        example_filter = pooling_filters[filter_idx, :]
        top_indices = np.argsort(example_filter)[::-1]
        filter_norm = np.max(np.abs(example_filter))
        for bf_idx in top_indices[:num_connected_weights]:
          connection_strength = example_filter[bf_idx]/filter_norm
          colorVal = scalarMap.to_rgba(connection_strength)
          plot_func(sub_axes[y_id, x_id], bf_stats, bf_idx, colorVal)
        sub_axes[y_id, x_id].set_xlim(0, 15)
        sub_axes[y_id, x_id].set_ylim(0, 15)
        sub_axes[y_id, x_id].set_aspect("equal")
        filter_total += 1
      else:
        sub_axes[y_id, x_id].spines["right"].set_color("none")
        sub_axes[y_id, x_id].spines["top"].set_color("none")
        sub_axes[y_id, x_id].spines["left"].set_color("none")
        sub_axes[y_id, x_id].spines["bottom"].set_color("none")
      sub_axes[y_id, x_id].invert_yaxis()
      sub_axes[y_id, x_id].set_yticklabels([])
      sub_axes[y_id, x_id].set_xticklabels([])
      sub_axes[y_id, x_id].set_aspect("equal")
      sub_axes[y_id, x_id].tick_params(axis="both", bottom="off", top="off",
        left="off", right="off")
  scalarMap._A = []
  cbar = fig.colorbar(scalarMap, ax=list(sub_axes[:, -1]), ticks=[-1, 0, 1])
  plt.show()

#### Render lines to represent BFs

In [ ]:
def plot_lines(axis, bf_stats, bf_idx, colorVal):
  y, x = bf_stats["orientations"][bf_idx]
  y_cen, x_cen = bf_stats["envelope_centers"][bf_idx]
  length = 3#bf_stats["lengths"][bf_idx][0]
  x_points = [(x_cen-x*length/2), (x_cen+x*length/2)]
  y_points = [(y_cen-y*length/2), (y_cen+y*length/2)]  
  axis.plot(x_points, y_points, color=colorVal)

#### Render oriented ellipse to represent BFs

In [ ]:
def plot_ellipse(axis, bf_stats, bf_idx, colorVal):
  y_cen, x_cen = bf_stats["envelope_centers"][bf_idx]
  (y_ang, x_ang) = bf_stats["orientations"][bf_idx]
  angle = np.rad2deg(np.arctan2(y_ang, x_ang))
  e = matplotlib.patches.Ellipse(xy=[x_cen, y_cen], width=0.8,
    height=0.3, angle=angle, color=colorVal, alpha=1.0, fill=True)
  axis.add_artist(e)
  e.set_clip_box(axis.bbox)

## Construct analysis plots

### Eigenvalues of activity covariance matrix

In [ ]:
evals, evecs = np.linalg.eig(a_cov)
fig, ax = plt.subplots(1, figsize=(10,10))
ax.semilogy(evals)
ax.set_xlim(1, 500) # Ignore first eigenvalue
ax.set_ylim(0, 1200)
ax.set_yscale("log")
plt.show()

### Basis function analysis

In [ ]:
plot_hilbert_analysis(weights, params)

In [ ]:
plot_bf_stats(weights, params) 

### Pooling and eigen summary plots

In [ ]:
num_pooling_filters = 50
num_connected_weights = 250 # Top connected weights

In [ ]:
plot_pooling_filter_images(params, weights, pooling_filters,
  num_connected_weights, num_pooling_filters)

In [ ]:
plot_pooling_func(plot_lines, params, weights, pooling_filters,
  num_connected_weights, num_pooling_filters)

In [ ]:
plot_pooling_func(plot_ellipse, params, weights, evecs[sort_indices[:50]],
  num_connected_weights, num_pooling_filters)

In [ ]:
plot_pooling_func(plot_ellipse, params, weights, pooling_filters,
  num_connected_weights, num_pooling_filters)